In [1]:
try:
    __IPYTHON__
    USING_IPYTHON = True
    %load_ext autoreload
    %autoreload 2
except NameError:
    USING_IPYTHON = False

#### Argparse

In [83]:
import argparse
ap = argparse.ArgumentParser()
ap.add_argument('project_root', help='')
ap.add_argument('--mrp-data-dir', default='data', help='')
ap.add_argument('--mrp-test-dir', default='src/tests', help='')
ap.add_argument('--allennlp-serialization-dir-template', default='output/allennlp-serialization-2019-07-22-{}', help='')
ap.add_argument('--tests-fixtures-file-template', default='fixtures/{}-test.jsonl', help='')

ap.add_argument('--graphviz-sub-dir', default='visualization/mrp-figures', help='')
ap.add_argument('--train-sub-dir', default='training', help='')
ap.add_argument('--train-companion-file', default='companion/udpipe.mrp', help='')
ap.add_argument('--companion-sub-dir', default='companion')
ap.add_argument('--jamr-alignment-file', default='jamr.mrp')

ap.add_argument('--evaluation-allennlp-mrp-json-file', default='evaluation/allennlp-evaluation.mrp')
ap.add_argument('--test-input-file', default='evaluation/input.mrp', help='')
ap.add_argument('--test-companion-file', default='evaluation/udpipe.mrp', help='')
ap.add_argument('--allennlp-cache-dir', default='allennlp-cache', help='')
ap.add_argument('--allennlp-mrp-json-file-template', default='allennlp-data-small/allennlp-mrp-json-big-{}-{}.jsonl', help='')
ap.add_argument('--data-size-limit', type=int, default=10000, help='')

ap.add_argument('--mrp-file-extension', default='.mrp')
ap.add_argument('--companion-file-extension', default='.conllu')
ap.add_argument('--graphviz-file-template', default='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/mrp-figures/{}/{}.png')
ap.add_argument('--parse-plot-file-template', default='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/mrp-figures/{}/{}.png')

ap.add_argument('--cuda-device', type=int, default=-1)

arg_string = """
    /data/proj29_ds1/home/slai/mrp2019
    --cuda-device 0
"""
arguments = [arg for arg_line in arg_string.split(r'\\n') for arg in arg_line.split()]

In [84]:
if USING_IPYTHON:
    args = ap.parse_args(arguments)
else:
    args = ap.parse_args()

In [85]:
args

Namespace(allennlp_cache_dir='allennlp-cache', allennlp_mrp_json_file_template='allennlp-data-small/allennlp-mrp-json-big-{}-{}.jsonl', allennlp_serialization_dir_template='output/allennlp-serialization-2019-07-22-{}', companion_file_extension='.conllu', companion_sub_dir='companion', cuda_device=0, data_size_limit=10000, evaluation_allennlp_mrp_json_file='evaluation/allennlp-evaluation.mrp', graphviz_file_template='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/mrp-figures/{}/{}.png', graphviz_sub_dir='visualization/mrp-figures', jamr_alignment_file='jamr.mrp', mrp_data_dir='data', mrp_file_extension='.mrp', mrp_test_dir='src/tests', parse_plot_file_template='http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/mrp-figures/{}/{}.png', project_root='/data/proj29_ds1/home/slai/mrp2019', test_companion_file='evaluation/udpipe.mrp', test_input_file='evaluation/input.mrp', tests_fixtures_file_template='fixtures/{}-test.jsonl', train_companion_file=

#### Library imports

In [5]:
import json
import logging
import os
import pprint
import re
import string
from collections import Counter, defaultdict, deque
from multiprocessing import Process

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import plot_util
import torch
from action_state import mrp_json2parser_states, _generate_parser_action_states
from action_state import ERROR, APPEND, RESOLVE, IGNORE
from preprocessing import (CompanionParseDataset, MrpDataset, JamrAlignmentDataset,
                           read_companion_parse_json_file, read_mrp_json_file, parse2parse_json)            
from torch import nn
from tqdm import tqdm

#### ipython notebook specific imports

In [6]:
if USING_IPYTHON:
    # matplotlib config
    %matplotlib inline

In [7]:
sh = logging.StreamHandler()
formatter = logging.Formatter('%(levelname)-8s [%(name)s:%(lineno)d] %(message)s')
sh.setFormatter(formatter)
logging.basicConfig(
    level=logging.DEBUG, 
    handlers=[sh]
)
mute_logger_names = ['allennlp.data.iterators.data_iterator']
for logger_name in mute_logger_names:
    logger = logging.getLogger(logger_name)  # pylint: disable=invalid-name
    logger.setLevel(logging.INFO)

logger = logging.getLogger(__name__)  # pylint: disable=invalid-name
logger.setLevel(logging.INFO)

### Constants

In [8]:
UNKWOWN = 'UNKWOWN'

### Load data

In [9]:
train_dir = os.path.join(args.project_root, args.mrp_data_dir, args.train_sub_dir)

In [10]:
mrp_dataset = MrpDataset()

In [11]:
frameworks, framework2cid2mrp_json = mrp_dataset.load_mrp_json_dir(
    train_dir, args.mrp_file_extension)

frameworks: 100%|██████████| 5/5 [00:16<00:00,  2.95s/it]t/s]


In [12]:
framework2cid2mrp_json.keys()

dict_keys(['ucca', 'psd', 'eds', 'dm', 'amr'])

In [13]:
total_mrp_json_counts = [(f, len(c2m)) for f, c2m in framework2cid2mrp_json.items()]

In [14]:
total_mrp_json_counts

[('ucca', 4939), ('psd', 35656), ('eds', 35656), ('dm', 35656), ('amr', 56236)]

### Data Preprocessing companion

In [15]:
train_companion_filename = os.path.join(args.project_root, args.mrp_data_dir, args.train_sub_dir, args.train_companion_file)

In [16]:
cid2train_parse = read_companion_parse_json_file(train_companion_filename)

In [17]:
len(cid2train_parse)

98290

### Load JAMR alignment data

In [18]:
jalignment_dataset = JamrAlignmentDataset()

In [19]:
cid2alignment = jalignment_dataset.load_jamr_alignment_file(os.path.join(
    args.project_root,
    args.mrp_data_dir,
    args.companion_sub_dir,
    args.jamr_alignment_file
))

In [20]:
len(cid2alignment)

56236

### Load testing data

In [21]:
test_input_filename = os.path.join(args.project_root, args.mrp_data_dir, args.test_input_file)
test_companion_filename = os.path.join(args.project_root, args.mrp_data_dir, args.test_companion_file)

In [22]:
test_cid2mrp_json = read_mrp_json_file(test_input_filename)
test_cid2parse = read_companion_parse_json_file(test_companion_filename)

In [23]:
len(test_cid2mrp_json), len(test_cid2parse)

(6288, 6288)

### Test mrp parse data matching

In [24]:
test_configs = [
    ('ucca', 70),
    ('psd', 3),
    ('dm', 3),
]
framework, idx = test_configs[-1]

In [25]:
cid = list(framework2cid2mrp_json[framework].items())[idx][0]
cid

'20003002'

In [26]:
mrp_json = framework2cid2mrp_json[framework][cid]

In [27]:
parse_json = cid2train_parse[cid]

In [28]:
assert mrp_json['input'] == parse_json['input']

In [29]:
logger.info(args.graphviz_file_template.format(
    framework, cid))

INFO     [__main__:2] http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/mrp-figures/dm/20003002.png


In [30]:
doc = mrp_json['input']

In [31]:
token_pos = 0
anchors = []
char_pos2tokenized_parse_node_id = []

for node_id, node in enumerate(parse_json.get('nodes')):
    label = node.get('label')
    label_size = len(label)
    while doc[token_pos] == ' ':
        token_pos += 1
        char_pos2tokenized_parse_node_id.append(node_id)
    anchors.append((token_pos, token_pos + label_size))
    char_pos2tokenized_parse_node_id.extend([node_id] * (label_size))
    print(node_id, doc[token_pos: token_pos + label_size], anchors[-1], len(char_pos2tokenized_parse_node_id))
    token_pos += label_size

0 The (0, 3) 3
1 asbestos (4, 12) 12
2 fiber (13, 18) 18
3 , (18, 19) 19
4 crocidolite (20, 31) 31
5 , (31, 32) 32
6 is (33, 35) 35
7 unusually (36, 45) 45
8 resilient (46, 55) 55
9 once (56, 60) 60
10 it (61, 63) 63
11 enters (64, 70) 70
12 the (71, 74) 74
13 lungs (75, 80) 80
14 , (80, 81) 81
15 with (82, 86) 86
16 even (87, 91) 91
17 brief (92, 97) 97
18 exposures (98, 107) 107
19 to (108, 110) 110
20 it (111, 113) 113
21 causing (114, 121) 121
22 symptoms (122, 130) 130
23 that (131, 135) 135
24 show (136, 140) 140
25 up (141, 143) 143
26 decades (144, 151) 151
27 later (152, 157) 157
28 , (157, 158) 158
29 researchers (159, 170) 170
30 said (171, 175) 175
31 . (175, 176) 176


In [32]:
len(char_pos2tokenized_parse_node_id)

176

In [33]:
doc = mrp_json['input']

In [34]:
mrp_json['tops']

[30]

In [35]:
status, mrp_parser_states, mrp_meta_data = mrp_json2parser_states(
    mrp_json, 
    tokenized_parse_nodes=parse_json['nodes'],
)

In [36]:
status

'success'

In [37]:
(
    doc,
    nodes,
    node_id2node,
    edge_id2edge,
    top_oriented_edges,
    token_nodes,
    # abstract_node_id_set,
    parent_id2indegree,
    # parent_id2child_id_set,
    # child_id2parent_id_set,
    # child_id2edge_id_set,
    # parent_id2edge_id_set,
    # parent_child_id2edge_id_set,
    parse_nodes_anchors,
    char_pos2tokenized_node_id,
    curr_node_ids,
    token_states,
    actions,
) = mrp_meta_data

In [38]:
curr_node_ids = mrp_meta_data[-3]
token_states = mrp_meta_data[-2]
actions = mrp_meta_data[-1]

In [39]:
*_, curr_node_ids, token_states, actions = mrp_meta_data

In [40]:
actions[1]

(1,
 (1,
  0,
  {'id': 0,
   'label': 'the',
   'properties': ['pos', 'frame'],
   'values': ['DT', 'q:i-h-h'],
   'anchors': [{'from': 0, 'to': 3}]},
  [[]]))

In [41]:
tokenized_parse_nodes = parse_json['nodes']

In [42]:
for curr_node_id, action, token_state in zip(curr_node_ids, actions, [[]] + token_states):
    action_type, params = action
    pprint.pprint((
        curr_node_id, 
        action[0], 
        tokenized_parse_nodes[curr_node_id]['values'][1],
        (action[1] or [None])[:2], [token_group[:4] for token_group in token_state],
    ))
                                                       

(0, 0, 'DET', [None], [])
(1, 1, 'NOUN', (1, 0), [(0, False, 'The', [])])
(1, 0, 'NOUN', [None], [(0, True, 'the', [(0, False, 'The', [])])])
(2,
 1,
 'NOUN',
 (1, 0),
 [(0, True, 'the', [(0, False, 'The', [])]), (1, False, 'asbestos', [])])
(2,
 0,
 'NOUN',
 [None],
 [(0, True, 'the', [(0, False, 'The', [])]),
  (1, True, 'asbestos', [(1, False, 'asbestos', [])])])
(3,
 2,
 'PUNCT',
 [None],
 [(0, True, 'the', [(0, False, 'The', [])]),
  (1, True, 'asbestos', [(1, False, 'asbestos', [])]),
  (2, False, 'fiber', [])])
(4,
 0,
 'NOUN',
 [None],
 [(0, True, 'the', [(0, False, 'The', [])]),
  (1, True, 'asbestos', [(1, False, 'asbestos', [])]),
  (2, False, 'fiber', [])])
(5,
 1,
 'PUNCT',
 (1, 0),
 [(0, True, 'the', [(0, False, 'The', [])]),
  (1, True, 'asbestos', [(1, False, 'asbestos', [])]),
  (2, False, 'fiber', []),
  (4, False, 'crocidolite', [])])
(5,
 1,
 'PUNCT',
 (4, 2),
 [(0, True, 'the', [(0, False, 'The', [])]),
  (1, True, 'asbestos', [(1, False, 'asbestos', [])]),
  (2, F

In [43]:
actions

[(0, None),
 (1,
  (1,
   0,
   {'id': 0,
    'label': 'the',
    'properties': ['pos', 'frame'],
    'values': ['DT', 'q:i-h-h'],
    'anchors': [{'from': 0, 'to': 3}]},
   [[]])),
 (0, None),
 (1,
  (1,
   0,
   {'id': 1,
    'label': 'asbestos',
    'properties': ['pos', 'frame'],
    'values': ['NN', 'n:x'],
    'anchors': [{'from': 4, 'to': 12}]},
   [[]])),
 (0, None),
 (2, None),
 (0, None),
 (1,
  (1,
   0,
   {'id': 4,
    'label': 'crocidolite',
    'properties': ['pos', 'frame'],
    'values': ['NN', 'n:x'],
    'anchors': [{'from': 20, 'to': 31}]},
   [[]])),
 (1,
  (4,
   2,
   {'id': 2,
    'label': 'fiber',
    'properties': ['pos', 'frame'],
    'values': ['NN', 'n:x'],
    'anchors': [{'from': 13, 'to': 18}]},
   [[{'source': 0,
      'target': 2,
      'label': 'BV',
      'id': 2,
      'parent': 2,
      'child': 0}],
    [{'source': 1,
      'target': 2,
      'label': 'compound',
      'id': 1,
      'parent': 2,
      'child': 1}],
    [],
    [{'source': 4,
    

In [44]:
token_states[1]

[(0, True, 'the', [(0, False, 'The', [])])]

In [45]:
# [n['label'] for n in parse_json['nodes']]

In [46]:
# token_states[-1]

In [48]:
status, companion_parser_states, companion_meta_data = mrp_json2parser_states(
    parse_json,
    mrp_doc=doc,
    tokenized_parse_nodes=parse_json['nodes'],
)
status

'success'

### Create training instance

In [49]:
# # Only ucca
# framework = 'ucca'
# ignore_framework_set = {'amr', 'dm', 'psd', 'eds'}
# dataset = 'wiki'
# ignore_dataset_set = {}

# # Only dm
# framework = 'dm'
# ignore_framework_set = {'amr', 'ucca', 'psd', 'eds'}
# dataset = 'wsj'
# ignore_dataset_set = {}

# # Only psd
# framework = 'psd'
# ignore_framework_set = {'amr', 'ucca', 'dm', 'eds'}
# dataset = 'wsj'
# ignore_dataset_set = {}

# Only eds
# framework = 'eds'
# ignore_framework_set = {'amr', 'ucca', 'dm', 'psd'}
# dataset = 'wsj'
# ignore_dataset_set = {}

# # Only psd
# framework = 'psd'
# ignore_framework_set = {'amr', 'ucca', 'dm', 'eds'}
# dataset = 'wsj'
# ignore_dataset_set = {}

# # Only AMR
# framework = 'amr'
# ignore_framework_set = {'psd', 'ucca', 'dm', 'eds'}
# dataset = 'wsj'
# ignore_dataset_set = {}

# # all 
# framework = 'ucca'
# ignore_framework_set = {}
# dataset = 'wiki'
# ignore_dataset_set = {}

# # ucca-dm
# framework = 'ucca'
# ignore_framework_set = {'amr', 'psd', 'eds'}
# dataset = 'wiki'
# ignore_dataset_set = {}

# all data 
framework = 'ucca'
ignore_framework_set = {}
dataset = 'wiki'
ignore_dataset_set = {}

In [50]:
frameworks = ['dm', 'psd', 'eds', 'ucca', 'amr']
# frameworks = ['ucca', 'dm']
# frameworks = ['amr']

In [51]:
framework_names = '-'.join([
    framework 
    for framework in frameworks 
    if framework not in ignore_framework_set
])
framework_names

'dm-psd-eds-ucca-amr'

In [52]:
allennlp_tests_fixtures_output_file = os.path.join(
    args.project_root, args.mrp_test_dir, args.tests_fixtures_file_template.format(framework_names))

allennlp_framework_train_output_file = os.path.join(
    args.project_root, args.mrp_data_dir, args.allennlp_mrp_json_file_template.format(framework_names, 'train'))

allennlp_framework_test_output_file = os.path.join(
    args.project_root, args.mrp_data_dir, args.allennlp_mrp_json_file_template.format(framework_names, 'test'))
allennlp_framework_train_output_file

'/data/proj29_ds1/home/slai/mrp2019/data/allennlp-data-small/allennlp-mrp-json-big-dm-psd-eds-ucca-amr-train.jsonl'

In [53]:
# Create tests fixture jsonl
fixture_combinations = [
#     ('ucca', 70),
    ('dm', 3)
] * 5

with open(allennlp_tests_fixtures_output_file, 'w') as wf:
    for framework, idx in fixture_combinations:
        cid = list(framework2cid2mrp_json[framework].items())[idx][0]
        mrp_json = framework2cid2mrp_json[framework][cid]
        doc = mrp_json.get('input')
        
        alignment = {}
        if framework == 'amr':
            alignment = cid2alignment[cid]  
        parse_json = cid2train_parse[cid]

        if parse_json:
            status, mrp_parser_states, mrp_meta_data = mrp_json2parser_states(
                mrp_json, 
                tokenized_parse_nodes=parse_json['nodes'],
                alignment=alignment,
            )
            status, companion_parser_states, companion_meta_data = mrp_json2parser_states(
                parse_json, 
                mrp_doc=doc,
                tokenized_parse_nodes=parse_json['nodes'],
            )

            data_instance = {
                'mrp_json': mrp_json,
                'parse_json': parse_json,
                'mrp_parser_states': mrp_parser_states,
                'mrp_meta_data': mrp_meta_data,
                'companion_parser_states': companion_parser_states,
                'companion_meta_data': companion_meta_data,
            }
            json_encoded_instance = json.dumps(data_instance)
            wf.write(json_encoded_instance + '\n')
            
status

'success'

In [54]:
train_mrp_cid_set = set([cid for cid2m in framework2cid2mrp_json.values() for cid in cid2m.keys()])
len(train_mrp_cid_set)

96744

In [55]:
train_parse_cid_set = set(cid2train_parse.keys())
len(train_parse_cid_set)

98290

In [56]:
assert not (train_mrp_cid_set - train_parse_cid_set)

In [57]:
def json_encoded_instance_generator(total_split, split_no, mrp_status_counter, parse_status_counter):
    for framework, cid, mrp_json in tqdm(mrp_dataset.mrp_json_generator(
        ignore_framework_set=ignore_framework_set,
        total_split=total_split,
        split_no=split_no,
    #                 data_size_limit=args.data_size_limit * 2
    )):
        doc = mrp_json.get('input')

        framework = mrp_json.get('framework')
        alignment = {}
        if framework == 'amr':
            alignment = cid2alignment[cid]  
        parse_json = cid2train_parse[cid]

        if parse_json:
            status, mrp_parser_states, mrp_meta_data = mrp_json2parser_states(
                mrp_json, 
                tokenized_parse_nodes=parse_json['nodes'],
                alignment=alignment,
            )
            mrp_status_counter[status] += 1
            status, companion_parser_states, companion_meta_data = mrp_json2parser_states(
                parse_json, 
                mrp_doc=doc,
                tokenized_parse_nodes=parse_json['nodes'],
            )
            parse_status_counter[status] += 1

            # Continue if error
            if not mrp_parser_states:
                continue

            data_instance = {
                'mrp_json': mrp_json,
                'parse_json': parse_json,
                'mrp_parser_states': mrp_parser_states,
                'mrp_meta_data': mrp_meta_data,
                'companion_parser_states': companion_parser_states,
                'companion_meta_data': companion_meta_data,
            }
            json_encoded_instance = json.dumps(data_instance)
            yield json_encoded_instance

In [58]:
def generate_json_encoded_instance_split(total_split, split_no, data_split_type='train'):
    mrp_status_counter = Counter()
    parse_status_counter = Counter()
    allennlp_framework_output_file = os.path.join(
        args.project_root, 
        args.mrp_data_dir, 
        args.allennlp_mrp_json_file_template.format(framework_names, data_split_type) + '.split' + str(split_no)
    )
    logger.info(allennlp_framework_output_file)
    with open(allennlp_framework_output_file, 'w') as wf:
        for json_encoded_instance in json_encoded_instance_generator(
            total_split, split_no, mrp_status_counter, parse_status_counter
        ):
            wf.write(json_encoded_instance + '\n')
    return mrp_status_counter, parse_status_counter

### split data to 40 splits for parallel processing

In [59]:
train_split = 30
dev_split = 10
total_split = train_split + dev_split

# Create train jsonl
if os.path.isfile(allennlp_framework_train_output_file) and os.path.isfile(
    allennlp_framework_train_output_file):
    logger.info('allennlp_train_output_file found, stop generation')
else:
    all_process = []
    for split_no in range(train_split):
        p = Process(target=generate_json_encoded_instance_split, args=(total_split, split_no, 'train'))
        p.start()
        all_process.append(p)
    
    for split_no in range(train_split, total_split):
        p = Process(target=generate_json_encoded_instance_split, args=(total_split, split_no, 'test'))
        p.start()
        all_process.append(p)
    
    for p in all_process:
        p.join()
#     pass
# if 1==1:

INFO     [__main__:8] allennlp_train_output_file found, stop generation


### Combine files

In [60]:
dirname = os.path.dirname(allennlp_framework_train_output_file)
dirname

'/data/proj29_ds1/home/slai/mrp2019/data/allennlp-data-small'

In [61]:
if not os.path.isfile(allennlp_framework_train_output_file):
    with open(allennlp_framework_train_output_file, 'w') as wf:
        for filename in tqdm(os.listdir(dirname)):
            if 'split' in filename:
                if 'train' in filename:
                    with open(os.path.join(dirname, filename)) as rf:
                        for line in rf:
                            wf.write(line)

In [62]:
if not os.path.isfile(allennlp_framework_test_output_file):
    with open(allennlp_framework_test_output_file, 'w') as wf:
        for filename in tqdm(os.listdir(dirname)):
            if 'split' in filename:
                if 'test' in filename:
                    with open(os.path.join(dirname, filename)) as rf:
                        for line in rf:
                            wf.write(line)

### Generate test mrp json data

In [63]:
allennlp_evaluation_output_file = os.path.join(
    args.project_root, args.mrp_data_dir, args.evaluation_allennlp_mrp_json_file)
allennlp_evaluation_output_file

'/data/proj29_ds1/home/slai/mrp2019/data/evaluation/allennlp-evaluation.mrp'

In [64]:
if os.path.isfile(allennlp_evaluation_output_file):
    logger.info('allennlp_evaluation_output_file found, stop generation')
else:
    with open(allennlp_evaluation_output_file, 'w') as wf:
        for mrp_json in tqdm(test_mrp_jsons):
            cid = mrp_json.get('id')
            doc = mrp_json.get('input')
            frameworks = mrp_json.get('targets')
            parse_json = test_parse_jsons[cid]

            for framework in frameworks:
                companion_parser_states, companion_meta_data = mrp_json2parser_states(
                    parse_json, 
                    mrp_doc=doc,
                    tokenized_parse_nodes=parse_json['nodes'],
                )

                data_instance = {
                    'cid': cid,
                    'doc': doc,
                    'framework': framework,
                    'mrp_json': mrp_json,
                    'parse_json': parse_json,
                    'companion_parser_states': companion_parser_states,
                    'companion_meta_data': companion_meta_data,
                }
                json_encoded_instance = json.dumps(data_instance)
                wf.write(json_encoded_instance + '\n')

INFO     [__main__:2] allennlp_evaluation_output_file found, stop generation


### Test allennlp dataset reader

In [65]:
import torch.optim as optim

from allennlp.common.file_utils import cached_path
from allennlp.data.vocabulary import Vocabulary
from allennlp.modules.text_field_embedders import TextFieldEmbedder, BasicTextFieldEmbedder
from allennlp.modules.token_embedders.embedding import Embedding
from allennlp.modules.seq2seq_encoders import Seq2SeqEncoder, PytorchSeq2SeqWrapper
from allennlp.modules.feedforward import FeedForward

from allennlp.training.metrics import CategoricalAccuracy

from allennlp.data.iterators import BucketIterator
from allennlp.training.trainer import Trainer

import json
import logging
from typing import Dict

from allennlp.common.file_utils import cached_path
from allennlp.data.dataset_readers.dataset_reader import DatasetReader
from allennlp.data.fields import LabelField, TextField
from allennlp.data.instance import Instance
from allennlp.data.token_indexers import SingleIdTokenIndexer, TokenIndexer
from allennlp.data.tokenizers import Token, Tokenizer, WordTokenizer
from allennlp.models import Model
from overrides import overrides

INFO     [pytorch_pretrained_bert.modeling:230] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
/home/slai/.pyenv/versions/mrp/lib/python3.7/site-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
DEBUG    [allennlp.common.registrable:56] instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
DEBUG    [allennlp.common.registrable:56] instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
DEBUG    [allennlp.common.registrable:56] instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>
DEBUG    [allennlp.common.registrable:56] instantiating registered subclass relu of <class 'allennlp.nn.activations.Activation'>


In [66]:
from mrp_library.dataset_readers.mrp_jsons_actions import MRPDatasetActionReader

In [67]:
reader = MRPDatasetActionReader()

In [68]:
allennlp_cache_dataset_dir = os.path.join(
    args.project_root, args.mrp_data_dir, args.allennlp_cache_dir)
allennlp_cache_dataset_dir

'/data/proj29_ds1/home/slai/mrp2019/data/allennlp-cache'

In [69]:
# reader.cache_data(allennlp_cache_dataset_dir)

In [70]:
train_dataset = reader.read(cached_path(allennlp_framework_train_output_file))

0it [00:00, ?it/s]WARNING  [allennlp.data.fields.label_field:72] Your label namespace was 'ucca_resolved_label_root_label'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.
6728it [00:02, 3293.91it/s]WARNING  [allennlp.data.fields.label_field:72] Your label namespace was 'psd_resolved_label_root_label'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.
231172it [01:43, 2238.32it/s]


In [71]:
# test_dataset = reader.read(cached_path(allennlp_train_output_file))
test_dataset = reader.read(cached_path(allennlp_framework_test_output_file))

94349it [00:55, 1709.51it/s]


In [72]:
tests_fixtures_dataset = reader.read(cached_path(allennlp_tests_fixtures_output_file))

0it [00:00, ?it/s]WARNING  [allennlp.data.fields.label_field:72] Your label namespace was 'dm_resolved_label_root_label'. We recommend you use a namespace ending with 'labels' or 'tags', so we don't add UNK and PAD tokens by default to your vocabulary.  See documentation for `non_padded_namespaces` parameter in Vocabulary.
385it [00:00, 3988.48it/s]


In [73]:
non_padded_namespaces = [
    'action_type_labels',
]
label_fields = [
    'resolved_label_root_label',
    'resolved_label_edge_labels',
]
for framework in frameworks:
    for label_field in label_fields:
        non_padded_namespaces.append('{}_{}'.format(framework, label_field))

In [74]:
non_padded_namespaces

['action_type_labels',
 'dm_resolved_label_root_label',
 'dm_resolved_label_edge_labels',
 'psd_resolved_label_root_label',
 'psd_resolved_label_edge_labels',
 'eds_resolved_label_root_label',
 'eds_resolved_label_edge_labels',
 'ucca_resolved_label_root_label',
 'ucca_resolved_label_edge_labels',
 'amr_resolved_label_root_label',
 'amr_resolved_label_edge_labels']

In [75]:
vocab = Vocabulary.from_instances(
    train_dataset + test_dataset + tests_fixtures_dataset,
    non_padded_namespaces=non_padded_namespaces,
)

INFO     [allennlp.data.vocabulary:396] Fitting token dictionary from dataset.
100%|██████████| 325906/325906 [00:26<00:00, 12532.28it/s]


In [76]:
vocab.print_statistics()

INFO     [allennlp.data.vocabulary:664] Printed vocabulary statistics are only for the part of the vocabulary generated from instances. If vocabulary is constructed by extending saved vocabulary with dataset instances, the directly loaded portion won't be considered here.




----Vocabulary Statistics----


Top 10 most frequent tokens in namespace 'word':
	Token: the		Frequency: 1104817
	Token: ,		Frequency: 1035042
	Token: .		Frequency: 852116
	Token: a		Frequency: 504428
	Token: of		Frequency: 453468
	Token: to		Frequency: 397070
	Token: in		Frequency: 387815
	Token: be		Frequency: 370039
	Token: and		Frequency: 241086
	Token: for		Frequency: 194916

Top 10 longest tokens in namespace 'word':
	Token: http://www.speedtest.net/result/1155244347.png		length: 46	Frequency: 12
	Token: http://cambridgefoodfrivolity.blogspot.com/		length: 43	Frequency: 12
	Token: http://en.wikipedia.org/wiki/bullfighting		length: 41	Frequency: 104
	Token: bioequivalence-therapeutic-equivalence		length: 38	Frequency: 110
	Token: www.norcalfightingalliance.com		length: 30	Frequency: 26
	Token: catalog-clothing-merchandiser		length: 29	Frequency: 140
	Token: research-and-production		length: 23	Frequency: 128
	Token: interest-rate-sensitive		length: 23	Frequency: 74
	Token: www.de

In [77]:
assert vocab.get_vocab_size('action_type_labels') == 3

In [78]:
namespaces = [
    'token_node_label',
    'word',
    'pos',
    'label',
]
for framework in frameworks:
    namespaces.append('{}_resolved_label_edge_labels'.format(framework))
    namespaces.append('{}_resolved_label_root_label'.format(framework))

In [79]:
for namespace in namespaces:
    logger.info((namespace, vocab.get_vocab_size(namespace)))

INFO     [__main__:2] ('token_node_label', 19862)
INFO     [__main__:2] ('word', 21589)
INFO     [__main__:2] ('pos', 71)
INFO     [__main__:2] ('label', 2)
INFO     [__main__:2] ('dm_resolved_label_edge_labels', 6)
INFO     [__main__:2] ('dm_resolved_label_root_label', 21)
INFO     [__main__:2] ('psd_resolved_label_edge_labels', 83)
INFO     [__main__:2] ('psd_resolved_label_root_label', 10968)
INFO     [__main__:2] ('eds_resolved_label_edge_labels', 0)
INFO     [__main__:2] ('eds_resolved_label_root_label', 0)
INFO     [__main__:2] ('ucca_resolved_label_edge_labels', 31)
INFO     [__main__:2] ('ucca_resolved_label_root_label', 16)
INFO     [__main__:2] ('amr_resolved_label_edge_labels', 0)
INFO     [__main__:2] ('amr_resolved_label_root_label', 0)


### Test model

In [80]:
from mrp_library.models.resolver import Resolver
from mrp_library.iterators.same_instance_type_framework_stack_len_iterator import SameInstanceTypeFrameworkStackLenIterator

from allennlp.nn import InitializerApplicator, RegularizerApplicator, util
from allennlp.nn.activations import Activation
from allennlp.common.params import Params
from allennlp.nn.util import get_text_field_mask, sequence_cross_entropy_with_logits
from allennlp.modules.seq2vec_encoders.pytorch_seq2vec_wrapper import PytorchSeq2VecWrapper
from allennlp.modules.seq2seq_encoders.pytorch_seq2seq_wrapper import PytorchSeq2SeqWrapper

In [86]:
if torch.cuda.is_available():
    cuda_device = args.cuda_device
else:
    cuda_device = -1
cuda_device

0

In [87]:
def _cuda(variable, cuda_device):
    if cuda_device != -1:
        variable = variable.cuda(cuda_device)
    return variable

### Define parameters

In [615]:
bidirectional = True
EMBEDDING_DIM = 100
HIDDEN_DIM = 50

In [616]:
# frameworks = ['dm', 'psd', 'eds', 'ucca', 'amr']
frameworks = ['amr', 'ucca', 'psd', 'dm', 'eds']

# field_types = ['word', 'pos', 
#                'ucca_word', 'ucca_pos',
#                'dm_word', 'dm_pos', 
#                'resolved', 'token_node_label', 'token_node_prev_action']

field_type2embedder = {}
field_type2seq2vec_encoder = {}
field_type2seq2seq_encoder = {}

In [687]:
framework2metrics = {}
for framework in frameworks:
    common_metric_name2metric_dict = {
        'action_type': {
            'action_type_accuracy': CategoricalAccuracy()
        },
        'action_num_pop': {
                "action_num_pop_accuracy": CategoricalAccuracy(),
        },
        'root_label_type': {
                "child_edges_type_accuracy": CategoricalAccuracy(),
        },
        'child_edge_type': {
                "root_label_type_accuracy": CategoricalAccuracy(),
        },
    }
    framework2metrics[framework] = {
        metric_name: {framework + '-' + k :v for k, v in metric_dict.items()}
        for metric_name, metric_dict in common_metric_name2metric_dict.items()
    }

In [688]:
glove_embedding_group2field_types = {
    'word': ['word', 'parse_curr_word', 'parse_prev_word', 'parse_next_word'],
    'token_node_label': ['token_node_label'],
}
for field in ['word']:
    for framework in frameworks:
        glove_embedding_group2field_types[field].append('resolve_{}_{}'.format(framework, field))

embedding_group2field_types = {
    'pos': ['pos', 'parse_curr_pos', 'parse_prev_pos', 'parse_next_pos'],
    'resolved': ['resolved', 'token_node_resolved'],
    'token_node_prev_action': ['action', 'token_node_prev_action'],
    'template': ['template'],
}
for field in ['pos']:
    for framework in frameworks:
        embedding_group2field_types[field].append('resolve_{}_{}'.format(framework, field))

seq2vec_group2field_types = {
    'parse_word': ['word', 'parse_curr_word', 'parse_prev_word', 'parse_next_word'],
    'parse_pos': ['pos', 'parse_curr_pos', 'parse_prev_pos', 'parse_next_pos'],
    'resolved': ['resolved', 'token_node_resolved'],
    'token_node_label': ['token_node_label'],
    'action': ['action', 'token_node_prev_action'],
}
for field in ['word', 'pos']:
    for framework in frameworks:
        full_name = 'resolve_{}_{}'.format(framework, field)
        seq2vec_group2field_types[full_name] = [full_name]

seq2seq_group2field_types = {
    'parse_word': ['word', 'parse_curr_word', 'parse_prev_word', 'parse_next_word'],
    'parse_pos': ['pos', 'parse_curr_pos', 'parse_prev_pos', 'parse_next_pos'],
    'resolved': ['resolved', 'token_node_resolved'],
    'token_node_label': ['token_node_label'],
    'action': ['action', 'token_node_prev_action'],
}
for field in ['word', 'pos']:
    for framework in frameworks:
        full_name = 'resolve_{}_{}'.format(framework, field)
        seq2seq_group2field_types[full_name] = [full_name]

In [689]:
embedding_group2field_types.keys()

dict_keys(['pos', 'resolved', 'token_node_prev_action', 'template'])

In [690]:
# Embedding.from_params(
#     vocab=vocab,
#     params = Params({
#         'vocab_namespace': embedding_group,
#         'pretrained_file': '../data/glove.6B.100d.txt',
#         'embedding_dim' : EMBEDDING_DIM,
#     })
# )

In [691]:
for embedding_group, field_types in glove_embedding_group2field_types.items():
    embedding = _cuda(
        Embedding.from_params(
            vocab=vocab,
            params = Params({
                'vocab_namespace': embedding_group,
                'pretrained_file': '../data/glove.6B.100d.txt',
                'embedding_dim' : EMBEDDING_DIM,
                "trainable": True,
            })
        ), cuda_device)
    logger.info((embedding_group, vocab.get_vocab_size(embedding_group)))
    embedder = BasicTextFieldEmbedder({embedding_group: embedding})
    for field_type in field_types:
        field_type2embedder[field_type] = embedder

for embedding_group, field_types in embedding_group2field_types.items():
    embedding = _cuda(
        Embedding(
            num_embeddings=vocab.get_vocab_size(embedding_group),
            embedding_dim=EMBEDDING_DIM
        ), cuda_device)
    logger.info((embedding_group, vocab.get_vocab_size(embedding_group)))
    embedder = BasicTextFieldEmbedder({embedding_group: embedding})
    for field_type in field_types:
        field_type2embedder[field_type] = embedder
    
for seq2vec_group, field_types in seq2vec_group2field_types.items():
    seq2vec = PytorchSeq2VecWrapper(
        _cuda(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True, bidirectional=bidirectional), cuda_device)
    )
    for field_type in field_types:
        field_type2seq2vec_encoder[field_type] = seq2vec

for seq2seq_group, field_types in seq2seq_group2field_types.items():
    seq2seq = PytorchSeq2SeqWrapper(
        _cuda(torch.nn.LSTM(EMBEDDING_DIM, HIDDEN_DIM, batch_first=True, bidirectional=bidirectional), cuda_device)
    )
    for field_type in field_types:
        field_type2seq2seq_encoder[field_type] = seq2seq

INFO     [allennlp.common.params:252] num_embeddings = None
INFO     [allennlp.common.params:252] vocab_namespace = word
INFO     [allennlp.common.params:252] embedding_dim = 100
INFO     [allennlp.common.params:252] pretrained_file = ../data/glove.6B.100d.txt
INFO     [allennlp.common.params:252] projection_dim = None
INFO     [allennlp.common.params:252] trainable = True
INFO     [allennlp.common.params:252] padding_index = None
INFO     [allennlp.common.params:252] max_norm = None
INFO     [allennlp.common.params:252] norm_type = 2.0
INFO     [allennlp.common.params:252] scale_grad_by_freq = False
INFO     [allennlp.common.params:252] sparse = False
INFO     [allennlp.modules.token_embedders.embedding:393] Reading pretrained embeddings from file








0it [00:00, ?it/s]







3215it [00:00, 32143.90it/s]







7160it [00:00, 34034.48it/s]







12447it [00:00, 38106.82it/s]







20575it [00:00, 45328.77it/s]







33609it [00:00, 56354.99it/s]







52926it [00:00, 71559.64

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Santa was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Tuesday was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Pacific was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Johnson was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token UAL was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Big was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Los was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Bush was not found in the 

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Senate was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token America was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Express was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Boston was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Kong was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Federal was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Electric was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token State was not fo

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Britain was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token South was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token W. was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Business was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Panama was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Icahn was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Columbia was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Allen was not found 

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Joseph was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Israel was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Services was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token S.A. was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token RU-486 was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Charles was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token County was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Lakers was not fou

...

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Stayed was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Seaview was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token TX was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token it. was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Incredible was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Summary was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Feel was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Number was not found i

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Novotel was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Cute was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Compare was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Bexar was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Bandera was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Natasha was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token blowdry was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Mackinaw was not 

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Aquarium was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token scrummy was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token HATED was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token BEFORE was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token STOP was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token EATTING was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Hancocks was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token windsheild was n

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token BOTHERED was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token ASWERING was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token absoulutely was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token ALSO was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token PEANUTS was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token FLOOR was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token arancini was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Ward was n

DEBUG    [allennlp.modules.token_embedders.embedding:439] Token TRANSPARENT was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Thoughts was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Eulogic was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token usally was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Come was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Cheese was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Nightmare was not found in the embedding file. Initialising randomly.
DEBUG    [allennlp.modules.token_embedders.embedding:439] Token Hobbs was 

In [692]:
field_type2embedder['word'].state_dict()['token_embedder_word.weight'].shape

torch.Size([21589, 100])

In [693]:
# word_embedding = Embedding(num_embeddings=vocab.get_vocab_size('word'),
#                             embedding_dim=EMBEDDING_DIM)
# pos_embedding = Embedding(num_embeddings=vocab.get_vocab_size('pos'),
#                             embedding_dim=EMBEDDING_DIM)

# word_embedder = BasicTextFieldEmbedder({
#     "word": word_embedding,
#     "pos": pos_embedding,
# })
# parse_label = {
#     'word': torch.LongTensor(
#         [
#             [ 1,  0,  3,  7,  2,  9,  4],
#             [ 0,  0,  5,  0,  0,  0,  4]
#         ]
#     ),
#     'pos': torch.LongTensor(
#         [
#             [ 1,  0,  3,  7,  2,  9,  4],
#             [ 0,  0,  5,  0,  0,  0,  4]
#         ]
#     )
# }

In [694]:
# embedded_parse_label = word_embedder(parse_label)

In [695]:
# embedded_parse_label.shape

In [696]:
vocab.get_vocab_size('resolved_label_edge_labels')

2

In [697]:
vocab.get_vocab_size('dm_resolved_label_edge_labels')

6

In [698]:
vocab.get_vocab_size('dm_resolved_label_root_label')

21

In [699]:
vocab.get_vocab_size('action_type_labels')

3

In [700]:
if bidirectional:
    LSTM_DIRECTIONS = 2
else:
    LSTM_DIRECTIONS = 1

In [701]:
vocab.get_vocab_size('action_type_labels')

3

In [702]:
frameworks

['amr', 'ucca', 'psd', 'dm', 'eds']

In [703]:
# action_classifier_params = Params({
#   "input_dim": HIDDEN_DIM * 15 * LSTM_DIRECTIONS + 4 * EMBEDDING_DIM,
#   "num_layers": 2,
#   "hidden_dims": [50, vocab.get_vocab_size('action_type_labels')],
#   "activations": ["sigmoid", "linear"],
#   "dropout": [0.0, 0.0]
# })

# action_num_pop_classifier_params = Params({
#   "input_dim": HIDDEN_DIM * 2 * LSTM_DIRECTIONS + EMBEDDING_DIM * 2,
#   "num_layers": 2,
#   "hidden_dims": [50, 1],
#   "activations": ["sigmoid", "linear"],
#   "dropout": [0.0, 0.0]
# })

# framework2field_type2feedforward_params = {}
# for framework in frameworks:
#     framework2field_type2feedforward_params[framework] = {
#         'child_edges': Params({
#             "input_dim": HIDDEN_DIM * 4 * LSTM_DIRECTIONS,
#             "num_layers": 2,
#             "hidden_dims": [100, vocab.get_vocab_size('{}_resolved_label_edge_labels'.format(framework))],
#             "activations": ["sigmoid", "linear"],
#             "dropout": [0.0, 0.0]
#         }),
#         'root_label': Params({
#             "input_dim": EMBEDDING_DIM * 4,
#             "num_layers": 2,
#             "hidden_dims": [100, vocab.get_vocab_size('{}_resolved_label_root_label'.format(framework))],
#             "activations": ["sigmoid", "linear"],
#             "dropout": [0.0, 0.0]
#         }),
#     }

In [784]:
framework2field_type2feedforward_params = {}
for framework in frameworks:
    framework2field_type2feedforward_params[framework] = {
        'child_edges': Params({
            "input_dim": HIDDEN_DIM * 4 * LSTM_DIRECTIONS,
            "num_layers": 2,
            "hidden_dims": [100, vocab.get_vocab_size('{}_resolved_label_edge_labels'.format(framework))],
            "activations": ["sigmoid", "linear"],
            "dropout": [0.0, 0.0]
        }),
        'root_label': Params({
            "input_dim": EMBEDDING_DIM * 4,
            "num_layers": 2,
            "hidden_dims": [100, vocab.get_vocab_size('{}_resolved_label_root_label'.format(framework))],
            "activations": ["sigmoid", "linear"],
            "dropout": [0.0, 0.0]
        }),
        'action': Params({
          "input_dim": HIDDEN_DIM * 15 * LSTM_DIRECTIONS + 4 * EMBEDDING_DIM,
          "num_layers": 2,
          "hidden_dims": [50, vocab.get_vocab_size('action_type_labels')],
          "activations": ["sigmoid", "linear"],
          "dropout": [0.0, 0.0]
        }),
        'action_num_pop': Params({
          "input_dim": HIDDEN_DIM * 2 * LSTM_DIRECTIONS + EMBEDDING_DIM * 2,
          "num_layers": 2,
          "hidden_dims": [50, 1],
          "activations": ["sigmoid", "linear"],
          "dropout": [0.0, 0.0]
        }),
    }

In [785]:
# action_classifier_feedforward = FeedForward.from_params(action_classifier_params)
# action_classifier_feedforward = _cuda(action_classifier_feedforward, cuda_device)

# action_num_pop_classifier_feedforward = FeedForward.from_params(action_num_pop_classifier_params)
# action_num_pop_classifier_feedforward = _cuda(action_num_pop_classifier_feedforward, cuda_device)

In [786]:
framework2field_type2feedforward = {}

for framework, field_type2feedforward_params in framework2field_type2feedforward_params.items():
    framework2field_type2feedforward[framework] = {}
    logger.info(framework)
    for field_type, feedforward_params in field_type2feedforward_params.items():
        logger.info(field_type)
        feedforward_classifier = FeedForward.from_params(
            feedforward_params)
        feedforward_classifier = _cuda(feedforward_classifier, cuda_device)
        framework2field_type2feedforward[framework][field_type] = feedforward_classifier
    framework2field_type2feedforward[framework] = torch.nn.ModuleDict(framework2field_type2feedforward[framework])

framework2field_type2feedforward = torch.nn.ModuleDict(framework2field_type2feedforward)

INFO     [__main__:5] psd
INFO     [__main__:7] child_edges
INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.modules.feedforward.FeedForward'> from params {'input_dim': 400, 'num_layers': 2, 'hidden_dims': [100, 83], 'activations': ['sigmoid', 'linear'], 'dropout': [0.0, 0.0]} and extras set()
INFO     [allennlp.common.params:252] input_dim = 400
INFO     [allennlp.common.params:252] num_layers = 2
INFO     [allennlp.common.params:252] hidden_dims = [100, 83]
INFO     [allennlp.common.params:252] hidden_dims = [100, 83]
INFO     [allennlp.common.params:252] activations = ['sigmoid', 'linear']
INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.nn.activations.Activation'> from params ['sigmoid', 'linear'] and extras set()
INFO     [allennlp.common.params:252] activations = ['sigmoid', 'linear']
INFO     [allennlp.common.from_params:340] instantiating class <class 'allennlp.nn.activations.Activation'> from params sigmoid and extr

In [787]:
field_type2embedder.keys()

dict_keys(['word', 'parse_curr_word', 'parse_prev_word', 'parse_next_word', 'resolve_amr_word', 'resolve_ucca_word', 'resolve_psd_word', 'resolve_dm_word', 'resolve_eds_word', 'token_node_label', 'pos', 'parse_curr_pos', 'parse_prev_pos', 'parse_next_pos', 'resolve_amr_pos', 'resolve_ucca_pos', 'resolve_psd_pos', 'resolve_dm_pos', 'resolve_eds_pos', 'resolved', 'token_node_resolved', 'action', 'token_node_prev_action', 'template'])

In [788]:
field_type2seq2vec_encoder.keys()

dict_keys(['word', 'parse_curr_word', 'parse_prev_word', 'parse_next_word', 'pos', 'parse_curr_pos', 'parse_prev_pos', 'parse_next_pos', 'resolved', 'token_node_resolved', 'token_node_label', 'action', 'token_node_prev_action', 'resolve_amr_word', 'resolve_ucca_word', 'resolve_psd_word', 'resolve_dm_word', 'resolve_eds_word', 'resolve_amr_pos', 'resolve_ucca_pos', 'resolve_psd_pos', 'resolve_dm_pos', 'resolve_eds_pos'])

In [789]:
framework2field_type2feedforward.keys()

odict_keys(['psd'])

In [791]:
field_type = 'word'

In [792]:
parse_label = {
    field_type: _cuda(
        torch.LongTensor(
            [
                [ 1,  0,  3,  7,  2,  9,  4],
                [ 0,  0,  5,  0,  0,  0,  4]
            ]
        ),
        cuda_device
    )
}
embedded_parse_label = field_type2embedder[field_type](parse_label)

In [793]:
feature_mask = util.get_text_field_mask(parse_label)

In [714]:
seq2vec_encoder = field_type2seq2vec_encoder[field_type]

In [715]:
encoded_feature = seq2vec_encoder(embedded_parse_label, feature_mask)

In [716]:
encoded_features = [encoded_feature] * 15 + [encoded_feature] * 4

In [717]:
# torch.cat(encoded_features, dim=-1)

In [718]:
actions

[(0, None),
 (1,
  (1,
   0,
   {'id': 0,
    'label': 'the',
    'properties': ['pos', 'frame'],
    'values': ['DT', 'q:i-h-h'],
    'anchors': [{'from': 0, 'to': 3}]},
   [[]])),
 (0, None),
 (1,
  (1,
   0,
   {'id': 1,
    'label': 'asbestos',
    'properties': ['pos', 'frame'],
    'values': ['NN', 'n:x'],
    'anchors': [{'from': 4, 'to': 12}]},
   [[]])),
 (0, None),
 (2, None),
 (0, None),
 (1,
  (1,
   0,
   {'id': 4,
    'label': 'crocidolite',
    'properties': ['pos', 'frame'],
    'values': ['NN', 'n:x'],
    'anchors': [{'from': 20, 'to': 31}]},
   [[]])),
 (1,
  (4,
   2,
   {'id': 2,
    'label': 'fiber',
    'properties': ['pos', 'frame'],
    'values': ['NN', 'n:x'],
    'anchors': [{'from': 13, 'to': 18}]},
   [[{'source': 0,
      'target': 2,
      'label': 'BV',
      'id': 2,
      'parent': 2,
      'child': 0}],
    [{'source': 1,
      'target': 2,
      'label': 'compound',
      'id': 1,
      'parent': 2,
      'child': 1}],
    [],
    [{'source': 4,
    

In [719]:
# logits = action_classifier_feedforward(torch.cat(encoded_features, dim=-1))

In [720]:
# logits.shape

In [721]:
label = torch.tensor([1, 0])

In [722]:
# loss_func = torch.nn.CrossEntropyLoss()
# loss = loss_func(logits, label)

In [723]:
Resolver = None

In [724]:
from mrp_library.models.resolver import Resolver
from mrp_library.iterators.same_instance_type_framework_stack_len_iterator import SameInstanceTypeFrameworkStackLenIterator


In [725]:
if torch.cuda.is_available():
    cuda_device = args.cuda_device
    model = Resolver(
        cuda_device=cuda_device,
        vocab=vocab,
        field_type2embedder=field_type2embedder,
        field_type2seq2vec_encoder=field_type2seq2vec_encoder,
        field_type2seq2seq_encoder=field_type2seq2seq_encoder,
        framework2field_type2feedforward=framework2field_type2feedforward,
        framework2metrics=framework2metrics,
    )
    model = model.cuda(cuda_device)
else:
    cuda_device = -1
    model = Resolver(
        cuda_device=cuda_device,
        vocab=vocab,
        field_type2embedder=field_type2embedder,
        field_type2seq2vec_encoder=field_type2seq2vec_encoder,
        field_type2seq2seq_encoder=field_type2seq2seq_encoder,
        framework2field_type2feedforward=framework2field_type2feedforward,
        framework2metrics=framework2metrics,
    )

# iterator = SameInstanceTypeFrameworkIterator(
#     shuffle=True,
#     batch_size=100, 
#     sorting_keys=[("token_node_resolveds", "num_tokens")],
# )
iterator = SameInstanceTypeFrameworkStackLenIterator(
    shuffle=True,
    batch_size=100, 
    sorting_keys=[("token_node_resolveds", "num_tokens")],
)
iterator.index_with(vocab)

optimizer = optim.SGD(model.parameters(), lr=0.1)

INFO     [allennlp.nn.initializers:293] Initializing parameters
INFO     [allennlp.nn.initializers:309] Done initializing parameters; the following parameters are using their default initialization from their code
INFO     [allennlp.nn.initializers:314]    field_type2embedder.action.token_embedder_token_node_prev_action.weight
INFO     [allennlp.nn.initializers:314]    field_type2embedder.parse_curr_pos.token_embedder_pos.weight
INFO     [allennlp.nn.initializers:314]    field_type2embedder.parse_curr_word.token_embedder_word.weight
INFO     [allennlp.nn.initializers:314]    field_type2embedder.resolved.token_embedder_resolved.weight
INFO     [allennlp.nn.initializers:314]    field_type2embedder.template.token_embedder_template.weight
INFO     [allennlp.nn.initializers:314]    field_type2embedder.token_node_label.token_embedder_token_node_label.weight
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.action._module.bias_hh_l0
INFO     [allennlp.nn.initializers:314] 

INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_eds_word._module.weight_hh_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_eds_word._module.weight_hh_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_eds_word._module.weight_ih_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_eds_word._module.weight_ih_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_psd_pos._module.bias_hh_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_psd_pos._module.bias_hh_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_psd_pos._module.bias_ih_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_psd_pos._module.bias_ih_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2seq_encoder.resolve_psd_pos._module.weight_hh_l0
INFO     

INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_amr_pos._module.bias_hh_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_amr_pos._module.bias_hh_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_amr_pos._module.bias_ih_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_amr_pos._module.bias_ih_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_amr_pos._module.weight_hh_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_amr_pos._module.weight_hh_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_amr_pos._module.weight_ih_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_amr_pos._module.weight_ih_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_amr_word._module.bias_hh_l0
INFO     [alle

INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_word._module.bias_ih_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_word._module.weight_hh_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_word._module.weight_hh_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_word._module.weight_ih_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolve_ucca_word._module.weight_ih_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolved._module.bias_hh_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolved._module.bias_hh_l0_reverse
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolved._module.bias_ih_l0
INFO     [allennlp.nn.initializers:314]    field_type2seq2vec_encoder.resolved._module.bias_ih_l0_reverse
INFO     [allennlp.nn.ini

INFO     [allennlp.nn.initializers:314]    framework2field_type2feedforward.psd.action_num_pop._linear_layers.0.weight
INFO     [allennlp.nn.initializers:314]    framework2field_type2feedforward.psd.action_num_pop._linear_layers.1.bias
INFO     [allennlp.nn.initializers:314]    framework2field_type2feedforward.psd.action_num_pop._linear_layers.1.weight
INFO     [allennlp.nn.initializers:314]    framework2field_type2feedforward.psd.child_edges._linear_layers.0.bias
INFO     [allennlp.nn.initializers:314]    framework2field_type2feedforward.psd.child_edges._linear_layers.0.weight
INFO     [allennlp.nn.initializers:314]    framework2field_type2feedforward.psd.child_edges._linear_layers.1.bias
INFO     [allennlp.nn.initializers:314]    framework2field_type2feedforward.psd.child_edges._linear_layers.1.weight
INFO     [allennlp.nn.initializers:314]    framework2field_type2feedforward.psd.root_label._linear_layers.0.bias
INFO     [allennlp.nn.initializers:314]    framework2field_type2feedforw

In [726]:
model.vocab.get_token_from_index(0, namespace='action_type_labels')

'RESOLVE'

In [817]:
cuda_device

0

In [818]:
model.resolve_tensor

tensor(0, device='cuda:0')

In [819]:
# list(model.named_parameters())

In [820]:
serialization_dir = os.path.join(args.project_root, args.allennlp_serialization_dir_template.format(framework_names))

In [731]:
trainer = Trainer(
    model=model,
    optimizer=optimizer,
    iterator=iterator,
    train_dataset=train_dataset,
    validation_dataset=test_dataset,
#     train_dataset=train_dataset,
#     validation_dataset=train_dataset,
    patience=10,
    num_epochs=20,
    cuda_device=cuda_device,
    histogram_interval=2300,
    serialization_dir=serialization_dir,
)

In [732]:
action_logits = torch.tensor([[-2.2126,  2.6022, -1.1655],
        [ 4.7340, -1.9992, -3.4521],
        [-1.9665,  2.4100, -1.2047],
        [-2.1353,  2.4847, -1.1260],
        [ 4.7492, -2.0234, -3.4460],
        [-1.4369,  1.9822, -1.2885],
        [ 1.0337,  0.3599, -2.0420],
        [ 5.0974, -2.3380, -3.4647],
        [ 5.4187, -2.4720, -3.6469],
        [-3.4045,  2.4903,  0.0773],
        [ 0.6384,  0.6764, -1.9942],
        [-2.2904,  2.7170, -1.2016],
        [ 4.6333, -1.9474, -3.4113],
        [-2.0811,  2.5367, -1.2174],
        [ 5.1840, -2.7536, -3.1499],
        [ 4.7421, -2.0138, -3.4485],
        [ 5.1121, -2.2290, -3.5999],
        [ 0.1843,  0.8324, -1.6990],
        [ 5.3854, -2.4593, -3.6309],
        [ 0.0324,  0.6715, -1.4173]])
action_logits = _cuda(action_logits, cuda_device)

action_type = torch.tensor([0, 2, 2, 1, 1, 0, 1, 2, 1, 2, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0])
action_type = _cuda(action_type, cuda_device)

In [733]:
action_probs, action_preds = action_logits.max(1)
action_resolve_preds = action_preds.eq_(model.resolve_tensor)

In [734]:
iter([1, 2, 3, 4])

In [735]:
defaultdict(lambda: defaultdict(dict))

defaultdict(<function __main__.<lambda>()>, {})

In [736]:
(action_resolve_preds, action_type, action_resolve_preds.eq(action_type))

(tensor([0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0],
        device='cuda:0'),
 tensor([0, 2, 2, 1, 1, 0, 1, 2, 1, 2, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0],
        device='cuda:0'),
 tensor([1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1],
        device='cuda:0', dtype=torch.uint8))

In [737]:
action_resolve_preds.unsqueeze(-1).float() * action_logits

tensor([[-0.0000,  0.0000, -0.0000],
        [ 4.7340, -1.9992, -3.4521],
        [-0.0000,  0.0000, -0.0000],
        [-0.0000,  0.0000, -0.0000],
        [ 4.7492, -2.0234, -3.4460],
        [-0.0000,  0.0000, -0.0000],
        [ 1.0337,  0.3599, -2.0420],
        [ 5.0974, -2.3380, -3.4647],
        [ 5.4187, -2.4720, -3.6469],
        [-0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.0000],
        [-0.0000,  0.0000, -0.0000],
        [ 4.6333, -1.9474, -3.4113],
        [-0.0000,  0.0000, -0.0000],
        [ 5.1840, -2.7536, -3.1499],
        [ 4.7421, -2.0138, -3.4485],
        [ 5.1121, -2.2290, -3.5999],
        [ 0.0000,  0.0000, -0.0000],
        [ 5.3854, -2.4593, -3.6309],
        [ 0.0000,  0.0000, -0.0000]], device='cuda:0')

In [738]:
embedded_fields = torch.ones(99, 62, 100)

In [739]:
embedded_fields.size()

torch.Size([99, 62, 100])

In [740]:
# embedded_fields

In [741]:
root_position_logits = torch.tensor([[0.1722, 0.1723, 0.1720, 0.1721, 0.1721, 0.1719, 0.1717, 0.1719, 0.1717,
         0.1716, 0.1719, 0.1717, 0.1716, 0.1718, 0.1719, 0.1718, 0.1719, 0.1717,
         0.1720, 0.1720, 0.1722, 0.1722, 0.1719, 0.1721, 0.1718, 0.1717, 0.1719,
         0.1717, 0.1720, 0.1715, 0.1718],
        [0.1722, 0.1723, 0.1720, 0.1721, 0.1721, 0.1719, 0.1717, 0.1719, 0.1717,
         0.1716, 0.1719, 0.1717, 0.1716, 0.1718, 0.1719, 0.1718, 0.1719, 0.1717,
         0.1720, 0.1720, 0.1722, 0.1722, 0.1719, 0.1721, 0.1718, 0.1717, 0.1719,
         0.1717, 0.1716, 0.1719, 0.1708],
        [0.1722, 0.1723, 0.1720, 0.1721, 0.1721, 0.1719, 0.1717, 0.1719, 0.1717,
         0.1716, 0.1719, 0.1717, 0.1716, 0.1718, 0.1719, 0.1718, 0.1719, 0.1717,
         0.1720, 0.1720, 0.1722, 0.1722, 0.1719, 0.1721, 0.1718, 0.1717, 0.1719,
         0.1717, 0.1720, 0.1715, 0.1717],
        [0.1722, 0.1723, 0.1720, 0.1721, 0.1721, 0.1719, 0.1717, 0.1719, 0.1717,
         0.1716, 0.1719, 0.1717, 0.1716, 0.1718, 0.1719, 0.1718, 0.1719, 0.1717,
         0.1720, 0.1720, 0.1722, 0.1722, 0.1719, 0.1721, 0.1718, 0.1717, 0.1719,
         0.1717, 0.1716, 0.1719, 0.1717],
        [0.1722, 0.1723, 0.1720, 0.1721, 0.1721, 0.1719, 0.1717, 0.1719, 0.1717,
         0.1716, 0.1719, 0.1717, 0.1716, 0.1718, 0.1719, 0.1718, 0.1719, 0.1717,
         0.1720, 0.1720, 0.1722, 0.1722, 0.1719, 0.1721, 0.1718, 0.1717, 0.1719,
         0.1717, 0.1720, 0.1715, 0.1709]])

In [742]:
root_position_logits.shape
root_position = torch.tensor([ 2,  0, -1,  1,  0])
mask = root_position.eq(torch.tensor(-1)).unsqueeze(-1).expand_as(root_position_logits)
mask = root_position.gt(-1)
masked_root_position_logits = root_position_logits * mask.float().unsqueeze(-1).expand_as(root_position_logits)
loss = torch.nn.CrossEntropyLoss()
inp = root_position_logits
# crossEntropy = -torch.log(torch.gather(inp, 1, root_position.view(-1, 1)))
root_position
# root_position.eq(torch.tensor(-2)).expand_as(root_position_logits)

tensor([ 2,  0, -1,  1,  0])

In [743]:
masked_root_position_logits.shape

torch.Size([5, 31])

In [744]:
root_position.shape

torch.Size([5])

In [745]:
torch.ones(2, 3, 47).transpose(1, 2).shape

torch.Size([2, 47, 3])

In [746]:
[dim for dim in torch.tensor([[]]).size()]

[1, 0]

In [747]:
torch.tensor([[]]).shape[1]

0

In [748]:
model.field_type2embedder.keys()

odict_keys(['action', 'parse_curr_pos', 'parse_curr_word', 'parse_next_pos', 'parse_next_word', 'parse_prev_pos', 'parse_prev_word', 'pos', 'resolve_amr_pos', 'resolve_amr_word', 'resolve_dm_pos', 'resolve_dm_word', 'resolve_eds_pos', 'resolve_eds_word', 'resolve_psd_pos', 'resolve_psd_word', 'resolve_ucca_pos', 'resolve_ucca_word', 'resolved', 'template', 'token_node_label', 'token_node_prev_action', 'token_node_resolved', 'word'])

In [749]:
import random

In [750]:
logging.ERROR

40

In [751]:
random.randint(0, 0)

0

In [795]:
metrics = framework2metrics['amr']

In [796]:
metrics

{'action_type': {'amr-action_type_accuracy': <allennlp.training.metrics.categorical_accuracy.CategoricalAccuracy at 0x7f5095645518>},
 'action_num_pop': {'amr-action_num_pop_accuracy': <allennlp.training.metrics.categorical_accuracy.CategoricalAccuracy at 0x7f5095645208>},
 'root_label_type': {'amr-child_edges_type_accuracy': <allennlp.training.metrics.categorical_accuracy.CategoricalAccuracy at 0x7f5095645080>},
 'child_edge_type': {'amr-root_label_type_accuracy': <allennlp.training.metrics.categorical_accuracy.CategoricalAccuracy at 0x7f5095645d30>}}

In [797]:
metric_dict = metrics['action_type']

In [798]:
metric_dict

{'amr-action_type_accuracy': <allennlp.training.metrics.categorical_accuracy.CategoricalAccuracy at 0x7f5095645518>}

In [799]:
trainer.train()

INFO     [allennlp.training.trainer:465] Beginning training.
INFO     [allennlp.training.trainer:281] Epoch 0/19
INFO     [allennlp.training.trainer:283] Peak CPU memory usage MB: 20599.34
INFO     [allennlp.training.trainer:287] GPU 0 memory usage MB: 10
INFO     [allennlp.training.trainer:287] GPU 1 memory usage MB: 1484
INFO     [allennlp.training.trainer:287] GPU 2 memory usage MB: 11
INFO     [allennlp.training.trainer:287] GPU 3 memory usage MB: 10
INFO     [allennlp.training.trainer:287] GPU 4 memory usage MB: 11
INFO     [allennlp.training.trainer:287] GPU 5 memory usage MB: 10
INFO     [allennlp.training.trainer:287] GPU 6 memory usage MB: 11
INFO     [allennlp.training.trainer:287] GPU 7 memory usage MB: 10
INFO     [allennlp.training.trainer:311] Training

  0%|          | 0/2312 [00:00<?, ?it/s]
amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000

amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-action_num_pop_accuracy: 0.2783, ucca-child_edges_type_accuracy: 0.3620, ucca-root_label_type_accuracy: 0.0826, psd-action_type_accuracy: 0.7585, psd-action_num_pop_accuracy: 0.1824, psd-child_edges_type_accuracy: 0.0324, psd-root_label_type_accuracy: 0.2327, dm-action_type_accuracy: 0.0000, dm-action_num_pop_accuracy: 0.0000, dm-child_edges_type_accuracy: 0.0000, dm-root_label_type_accuracy: 0.0000, eds-action_type_accuracy: 0.0000, eds-action_num_pop_accuracy: 0.0000, eds-child_edges_type_accuracy: 0.0000, eds-root_label_type_accuracy: 0.0000, loss: 3.3367 ||:   0%|          | 10/2312 [00:02<07:23,  5.19it/s]
amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-actio

amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-action_num_pop_accuracy: 0.2850, ucca-child_edges_type_accuracy: 0.3620, ucca-root_label_type_accuracy: 0.0826, psd-action_type_accuracy: 0.7585, psd-action_num_pop_accuracy: 0.1824, psd-child_edges_type_accuracy: 0.0320, psd-root_label_type_accuracy: 0.2233, dm-action_type_accuracy: 0.0000, dm-action_num_pop_accuracy: 0.0000, dm-child_edges_type_accuracy: 0.0000, dm-root_label_type_accuracy: 0.0000, eds-action_type_accuracy: 0.0000, eds-action_num_pop_accuracy: 0.0000, eds-child_edges_type_accuracy: 0.0000, eds-root_label_type_accuracy: 0.0000, loss: 2.8304 ||:   1%|          | 21/2312 [00:04<04:28,  8.55it/s]
amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-actio


amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-action_num_pop_accuracy: 0.3021, ucca-child_edges_type_accuracy: 0.3620, ucca-root_label_type_accuracy: 0.0826, psd-action_type_accuracy: 0.7585, psd-action_num_pop_accuracy: 0.1824, psd-child_edges_type_accuracy: 0.0320, psd-root_label_type_accuracy: 0.2233, dm-action_type_accuracy: 0.0000, dm-action_num_pop_accuracy: 0.0000, dm-child_edges_type_accuracy: 0.0000, dm-root_label_type_accuracy: 0.0000, eds-action_type_accuracy: 0.0000, eds-action_num_pop_accuracy: 0.0000, eds-child_edges_type_accuracy: 0.0000, eds-root_label_type_accuracy: 0.0000, loss: 2.2417 ||:   1%|▏         | 31/2312 [00:05<05:47,  6.56it/s]
amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-acti

amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-action_num_pop_accuracy: 0.3025, ucca-child_edges_type_accuracy: 0.3620, ucca-root_label_type_accuracy: 0.0826, psd-action_type_accuracy: 0.7585, psd-action_num_pop_accuracy: 0.1824, psd-child_edges_type_accuracy: 0.0320, psd-root_label_type_accuracy: 0.2232, dm-action_type_accuracy: 0.0000, dm-action_num_pop_accuracy: 0.0000, dm-child_edges_type_accuracy: 0.0000, dm-root_label_type_accuracy: 0.0000, eds-action_type_accuracy: 0.0000, eds-action_num_pop_accuracy: 0.0000, eds-child_edges_type_accuracy: 0.0000, eds-root_label_type_accuracy: 0.0000, loss: 2.1070 ||:   2%|▏         | 45/2312 [00:07<04:25,  8.53it/s]
amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-actio

...

amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-action_num_pop_accuracy: 0.3079, ucca-child_edges_type_accuracy: 0.3620, ucca-root_label_type_accuracy: 0.0826, psd-action_type_accuracy: 0.7672, psd-action_num_pop_accuracy: 0.1860, psd-child_edges_type_accuracy: 0.0309, psd-root_label_type_accuracy: 0.2305, dm-action_type_accuracy: 0.0000, dm-action_num_pop_accuracy: 0.0000, dm-child_edges_type_accuracy: 0.0000, dm-root_label_type_accuracy: 0.0000, eds-action_type_accuracy: 0.0000, eds-action_num_pop_accuracy: 0.0000, eds-child_edges_type_accuracy: 0.0000, eds-root_label_type_accuracy: 0.0000, loss: 1.7590 ||:  13%|█▎        | 292/2312 [00:39<03:47,  8.88it/s]
amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-acti

amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-action_num_pop_accuracy: 0.3079, ucca-child_edges_type_accuracy: 0.3620, ucca-root_label_type_accuracy: 0.0826, psd-action_type_accuracy: 0.7672, psd-action_num_pop_accuracy: 0.1860, psd-child_edges_type_accuracy: 0.0304, psd-root_label_type_accuracy: 0.2348, dm-action_type_accuracy: 0.0000, dm-action_num_pop_accuracy: 0.0000, dm-child_edges_type_accuracy: 0.0000, dm-root_label_type_accuracy: 0.0000, eds-action_type_accuracy: 0.0000, eds-action_num_pop_accuracy: 0.0000, eds-child_edges_type_accuracy: 0.0000, eds-root_label_type_accuracy: 0.0000, loss: 1.7725 ||:  13%|█▎        | 302/2312 [00:40<03:49,  8.75it/s]
amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-acti

amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-action_num_pop_accuracy: 0.3079, ucca-child_edges_type_accuracy: 0.3620, ucca-root_label_type_accuracy: 0.0826, psd-action_type_accuracy: 0.7672, psd-action_num_pop_accuracy: 0.1860, psd-child_edges_type_accuracy: 0.0299, psd-root_label_type_accuracy: 0.2389, dm-action_type_accuracy: 0.0000, dm-action_num_pop_accuracy: 0.0000, dm-child_edges_type_accuracy: 0.0000, dm-root_label_type_accuracy: 0.0000, eds-action_type_accuracy: 0.0000, eds-action_num_pop_accuracy: 0.0000, eds-child_edges_type_accuracy: 0.0000, eds-root_label_type_accuracy: 0.0000, loss: 1.7838 ||:  13%|█▎        | 312/2312 [00:41<03:48,  8.76it/s]
amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-acti

amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-action_num_pop_accuracy: 0.3079, ucca-child_edges_type_accuracy: 0.3620, ucca-root_label_type_accuracy: 0.0826, psd-action_type_accuracy: 0.7672, psd-action_num_pop_accuracy: 0.1860, psd-child_edges_type_accuracy: 0.0295, psd-root_label_type_accuracy: 0.2425, dm-action_type_accuracy: 0.0000, dm-action_num_pop_accuracy: 0.0000, dm-child_edges_type_accuracy: 0.0000, dm-root_label_type_accuracy: 0.0000, eds-action_type_accuracy: 0.0000, eds-action_num_pop_accuracy: 0.0000, eds-child_edges_type_accuracy: 0.0000, eds-root_label_type_accuracy: 0.0000, loss: 1.7965 ||:  14%|█▍        | 322/2312 [00:43<03:40,  9.04it/s]
amr-action_type_accuracy: 0.0000, amr-action_num_pop_accuracy: 0.0000, amr-child_edges_type_accuracy: 0.0000, amr-root_label_type_accuracy: 0.0000, ucca-action_type_accuracy: 0.0000, ucca-acti

KeyboardInterrupt: 

### Predictor

In [800]:
for metric in model.framework2metrics[framework]['action_num_pop'].values():
    pass

In [801]:
frameworks

['psd']

In [802]:
from mrp_library.predictors.mrp_predictor import MrpPredictor

In [803]:
reader = MRPDatasetActionReader()

In [804]:
mrp_predictor = MrpPredictor(model=model, dataset_reader=reader)

In [805]:
mrp_predictor.load_vocab(vocab)

In [806]:
use_test = True
use_test = False
framework = 'psd'

In [813]:
idx = 1
if use_test:
    cid = list(test_cid2mrp_json.keys())[idx]
    mrp_json = test_cid2mrp_json[cid]
    parse_json = test_cid2parse[cid]
    frameworks = mrp_json.get('targets')
    assert framework in frameworks
else:
    cid = list(framework2cid2mrp_json[framework].keys())[idx]
    mrp_json = framework2cid2mrp_json[framework][cid]
    parse_json = cid2train_parse[cid]

doc = mrp_json.get('input')

status, companion_parser_states, companion_meta_data = mrp_json2parser_states(
    parse_json, 
    mrp_doc=doc,
    tokenized_parse_nodes=parse_json['nodes'],
)

data_instance = {
    'cid': cid,
    'doc': doc,
    'framework': framework,
    'mrp_json': mrp_json,
    'parse_json': parse_json,
    'companion_parser_states': companion_parser_states,
    'companion_meta_data': companion_meta_data,
}

In [814]:
logger.info(args.graphviz_file_template.format(framework, cid))

INFO     [__main__:1] http://localhost:8000/files/proj29_ds1/home/slai/mrp2019/visualization/mrp-figures/psd/20001002.png


In [815]:
mrp_output_dict = mrp_predictor.predict_json(data_instance)

ERROR    [mrp_library.predictors.mrp_predictor:111] ('curr_state', {'mode': 'predict', 'curr_node_id': 1, 'framework': 'psd', 'token_state': [(0, False, 'Mr.', [])], 'parse_node_field_name2features': {'label': [mr., vinken, is, chairman, of, elsevier, n.v., ,, the, dutch, publishing, group, .], 'lemma': [Mr., Vinken, be, chairman, of, Elsevier, N.V., ,, the, dutch, publishing, group, .], 'upos': [PROPN, PROPN, VERB, NOUN, ADP, PROPN, PROPN, PUNCT, DET, ADJ, NOUN, NOUN, PUNCT], 'xpos': [NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, JJ, NN, NN, .]}, 'parse_node_state_field_name2features': {'curr_label': [vinken], 'curr_lemma': [Vinken], 'curr_upos': [PROPN], 'curr_xpos': [NNP], 'prev_label': [mr.], 'prev_lemma': [Mr.], 'prev_upos': [PROPN], 'prev_xpos': [NNP], 'next_label': [is, chairman, of, elsevier, n.v.], 'next_lemma': [be, chairman, of, Elsevier, N.V.], 'next_upos': [VERB, NOUN, ADP, PROPN, PROPN], 'next_xpos': [VBZ, NN, IN, NNP, NNP]}, 'token_node_field_name2features': {'resolved': [UNRE

('token_stack', [(0, False, 'Mr.', [])])
('token_stack', [(0, True, 'Mr.', [])])
('token_stack', [(0, True, 'Mr.', []), (1, False, 'Vinken', [])])
('token_stack', [(0, True, 'Mr.', []), (1, True, 'Vinken', [])])
('token_stack',
 [(0, True, 'Mr.', []), (1, True, 'Vinken', []), (2, False, 'is', [])])
('token_stack',
 [(0, True, 'Mr.', []), (1, True, 'Vinken', []), (2, True, 'is', [])])
('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3, False, 'chairman', [])])
('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3, False, 'chairman', [])])
('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3, False, 'chairman', []),
  (5, False, 'Elsevier', [])])
('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3, False, 'chairman', []),
  (5, True, 'Elsevier', [])])


ERROR    [mrp_library.predictors.mrp_predictor:111] ('curr_state', {'mode': 'predict', 'curr_node_id': 7, 'framework': 'psd', 'token_state': [(0, True, 'Mr.', []), (1, True, 'Vinken', []), (2, True, 'is', []), (3, False, 'chairman', []), (5, True, 'Elsevier', []), (6, False, 'N.V.', [])], 'parse_node_field_name2features': {'label': [mr., vinken, is, chairman, of, elsevier, n.v., ,, the, dutch, publishing, group, .], 'lemma': [Mr., Vinken, be, chairman, of, Elsevier, N.V., ,, the, dutch, publishing, group, .], 'upos': [PROPN, PROPN, VERB, NOUN, ADP, PROPN, PROPN, PUNCT, DET, ADJ, NOUN, NOUN, PUNCT], 'xpos': [NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, JJ, NN, NN, .]}, 'parse_node_state_field_name2features': {'curr_label': [,], 'curr_lemma': [,], 'curr_upos': [PUNCT], 'curr_xpos': [,], 'prev_label': [mr., vinken, is, chairman, of], 'prev_lemma': [Mr., Vinken, be, chairman, of], 'prev_upos': [PROPN, PROPN, VERB, NOUN, ADP], 'prev_xpos': [NNP, NNP, VBZ, NN, IN], 'next_label': [the, dutch, publ

ERROR    [mrp_library.predictors.mrp_predictor:111] ('curr_state', {'mode': 'predict', 'curr_node_id': 11, 'framework': 'psd', 'token_state': [(0, True, 'Mr.', []), (1, True, 'Vinken', []), (2, True, 'is', []), (3, False, 'chairman', []), (5, True, 'Elsevier', []), (6, True, 'N.V.', []), (9, False, 'Dutch', []), (10, True, 'publishing', [])], 'parse_node_field_name2features': {'label': [mr., vinken, is, chairman, of, elsevier, n.v., ,, the, dutch, publishing, group, .], 'lemma': [Mr., Vinken, be, chairman, of, Elsevier, N.V., ,, the, dutch, publishing, group, .], 'upos': [PROPN, PROPN, VERB, NOUN, ADP, PROPN, PROPN, PUNCT, DET, ADJ, NOUN, NOUN, PUNCT], 'xpos': [NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, JJ, NN, NN, .]}, 'parse_node_state_field_name2features': {'curr_label': [group], 'curr_lemma': [group], 'curr_upos': [NOUN], 'curr_xpos': [NN], 'prev_label': [mr., vinken, is, chairman, of], 'prev_lemma': [Mr., Vinken, be, chairman, of], 'prev_upos': [PROPN, PROPN, VERB, NOUN, ADP], 'prev_

('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3, False, 'chairman', []),
  (5, True, 'Elsevier', []),
  (6, False, 'N.V.', [])])
('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3, False, 'chairman', []),
  (5, True, 'Elsevier', []),
  (6, True, 'N.V.', [])])
('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3, False, 'chairman', []),
  (5, True, 'Elsevier', []),
  (6, True, 'N.V.', [])])
('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3, False, 'chairman', []),
  (5, True, 'Elsevier', []),
  (6, True, 'N.V.', [])])
('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3, False, 'chairman', []),
  (5, True, 'Elsevier', []),
  (6, True, 'N.V.', []),
  (9, False, 'Dutch', [])])
('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3, F

ERROR    [mrp_library.predictors.mrp_predictor:111] ('curr_state', {'mode': 'predict', 'curr_node_id': 12, 'framework': 'psd', 'token_state': [(0, True, 'Mr.', []), (1, True, 'Vinken', []), (2, True, 'is', []), (3, False, 'chairman', []), (5, True, 'Elsevier', []), (6, True, 'N.V.', []), (9, False, 'Dutch', []), (10, True, 'publishing', []), (11, True, 'group', [])], 'parse_node_field_name2features': {'label': [mr., vinken, is, chairman, of, elsevier, n.v., ,, the, dutch, publishing, group, .], 'lemma': [Mr., Vinken, be, chairman, of, Elsevier, N.V., ,, the, dutch, publishing, group, .], 'upos': [PROPN, PROPN, VERB, NOUN, ADP, PROPN, PROPN, PUNCT, DET, ADJ, NOUN, NOUN, PUNCT], 'xpos': [NNP, NNP, VBZ, NN, IN, NNP, NNP, ,, DT, JJ, NN, NN, .]}, 'parse_node_state_field_name2features': {'curr_label': [.], 'curr_lemma': [.], 'curr_upos': [PUNCT], 'curr_xpos': [.], 'prev_label': [mr., vinken, is, chairman, of], 'prev_lemma': [Mr., Vinken, be, chairman, of], 'prev_upos': [PROPN, PROPN, VERB, N

('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3, False, 'chairman', []),
  (5, True, 'Elsevier', []),
  (6, True, 'N.V.', []),
  (9, False, 'Dutch', []),
  (10, True, 'publishing', []),
  (11, True, 'group', [])])
('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3, False, 'chairman', []),
  (5, True, 'Elsevier', []),
  (6, True, 'N.V.', []),
  (9,
   True,
   'Dutch',
   [(9, False, 'Dutch', []),
    (10, True, 'publishing', []),
    (11, True, 'group', [])])])
('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3,
   True,
   'chairman',
   [(3, False, 'chairman', []),
    (5, True, 'Elsevier', []),
    (6, True, 'N.V.', []),
    (9,
     True,
     'Dutch',
     [(9, False, 'Dutch', []),
      (10, True, 'publishing', []),
      (11, True, 'group', [])])])])
('token_stack',
 [(0, True, 'Mr.', []),
  (1, True, 'Vinken', []),
  (2, True, 'is', []),
  (3,
 

In [816]:
mrp_output_dict

{'id': '20001002',
 'framework': 'psd',
 'input': 'Mr. Vinken is chairman of Elsevier N.V., the Dutch publishing group.',
 'nodes': [{'id': 0,
   'node_label': 'Mr.',
   'label': '#PersPron',
   'anchors': [{'from': 0, 'to': 3}]},
  {'id': 1,
   'node_label': 'Vinken',
   'label': '#PersPron',
   'anchors': [{'from': 4, 'to': 10}]},
  {'id': 2,
   'node_label': 'is',
   'label': '#PersPron',
   'anchors': [{'from': 11, 'to': 13}]},
  {'id': 3,
   'node_label': 'chairman',
   'label': '#PersPron',
   'anchors': [{'from': 14, 'to': 22}]},
  {'id': 5,
   'node_label': 'Elsevier',
   'label': '#PersPron',
   'anchors': [{'from': 26, 'to': 34}]},
  {'id': 6,
   'node_label': 'N.V.',
   'label': '#PersPron',
   'anchors': [{'from': 35, 'to': 39}]},
  {'id': 9,
   'node_label': 'Dutch',
   'label': '#PersPron',
   'anchors': [{'from': 45, 'to': 50}]},
  {'id': 10,
   'node_label': 'publishing',
   'label': '#PersPron',
   'anchors': [{'from': 51, 'to': 61}]},
  {'id': 11,
   'node_label': 'gr

In [811]:
token_stack = [(0, False, 'I', []), (1, False, 'was', []), (2, False, 'very', []), (3, False, 'much', []), (4, False, 'worried', []), (5, False, ',', []), (6, False, 'for', []), (7, False, 'it', []), (8, False, 'was', []), (9, False, 'becoming', []), (10, False, 'clear', []), (11, False, 'to', []), (12, False, 'me', []), (13, False, 'that', []), (14, False, 'the', []), (15, False, 'breakdown', []), (16, False, 'of', []), (17, False, 'my', []), (18, False, 'plane', []), (19, False, 'was', []), (20, False, 'extremely', []), (21, True, 'serious', []), (22, True, 'I', [])]


In [812]:
token_stack = [(0, True, 'I', []), (1, True, 'was', []), (2, True, 'very', []), ]

In [342]:
masked_action_num_pop_pred = None

In [343]:
not_resolved_exist = False
resolved_end_found = False
action_num_pop = 0
while all([
        action_num_pop < len(token_stack),
        not resolved_end_found,
    (not not_resolved_exist
     or (masked_action_num_pop_pred
         and action_num_pop < masked_action_num_pop_pred)),
]):
    token_node = token_stack[::-1][action_num_pop]
    print(token_node)
    (resolved_node_id, is_resolved, resolved_node_label,
     resolved_node_childs) = token_node
    logger.debug(('token_node', token_node))
    if not is_resolved:
        if not_resolved_exist:
            resolved_end_found = True
        else:
            not_resolved_exist = True
    if not resolved_end_found:
        action_num_pop += 1
    logger.debug(('token_node', action_num_pop, is_resolved,
                  resolved_end_found))
action_num_pop

(22, True, 'I', [])
(21, True, 'serious', [])
(20, False, 'extremely', [])


3

In [ ]:
vocab.get_token_from_index(4, namespace='ucca_resolved_label_root_label')

In [165]:
vocab.get_token_from_index(0, namespace='action_type_labels')

'RESOLVE'

In [ ]:
model.vocab

In [ ]:
vocab.get_token_index('RESOLVE')

In [ ]:
model.resolve_tensor

In [ ]:
mrp_output_dict

In [ ]:
model.framework2field_type2feedforward['eds']['child_edges']

In [ ]:
[1, 2, 3][-3]

In [ ]:
mrp_json

In [ ]:
torch.ones(98, 29, 5).squeeze(-1)

In [ ]:
torch.ones(98, 29, 5).view(-1, 29).shape

In [ ]:
vocab.get_token_index()

In [ ]:
vocab.get_token_from_index(2, namespace='action_type')

In [ ]:
vocab.get_token_from_index(1, namespace='resolve_label_root_label')

In [ ]:
token_state = [[26, True, 'H', [[23, True, 'A', [[0, True, 'E', [[0, False, 'The', []]]], [1, True, 'C', [[1, False, 'Lakers', []]]]]], [2, True, 'P', [[2, False, 'advanced', []]]], [25, True, 'A', [[3, True, 'R', [[3, False, 'through', []]]], [4, True, 'E', [[4, False, 'the', []]]], [24, True, 'P', [[5, True, 'T', [[5, False, '1982', []]]], [6, True, 'C', [[6, False, 'playoffs', []]]]]]]]]], [7, True, 'L', [[7, False, 'and', []]]], [8, True, 'P', [[8, False, 'faced', []]]], [9, True, 'A', [[9, False, 'Philadelphia', []]]], [27, True, 'D', [[10, True, 'R', [[10, False, 'for', []]]], [11, True, 'E', [[11, False, 'the', []]]], [12, True, 'Q', [[12, False, 'second', []]]], [13, True, 'C', [[13, False, 'time', []]]]]], [14, True, 'R', [[14, False, 'in', []]]], [15, True, 'Q', [[15, False, 'three', []]]], [16, False, 'years', []]]

In [ ]:
pprint.pprint(token_state)

In [ ]:
vocab.get_token_from_index(0, namespace='labels')

In [ ]:
vocab.get_token_index('RESOLVE')

In [ ]:
vocab.get_token_from_index(3, namespace='resolved')

In [ ]:
vocab.get_token_from_index(5, namespace='token_node_prev_action')